In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load Dataset

In [56]:
from hcmus.core import pconf
from hcmus.lbs import LabelStudioConnector
from hcmus.data import LbsDataset
from hcmus.utils import viz_utils

In [43]:
product_connector = LabelStudioConnector(
    url=pconf.LABEL_STUDIO_URL,
    api_key=pconf.LABEL_STUDIO_API_KEY,
    project_id=pconf.LABEL_STUDIO_PROJECT_ID,
)

In [66]:
from torchvision import transforms as T
product_dataset = LbsDataset(product_connector, augmentation=True)

Loading tasks:   4%|▍         | 4/100 [00:07<02:55,  1.83s/it]
2025-03-22 20:35:36.774 | INFO     | hcmus.data._lbs_dataset:__init__:31 - Number of labels: 203
2025-03-22 20:35:36.774 | INFO     | hcmus.data._lbs_dataset:__init__:32 - Number of data points: 1591


In [103]:
tensor, target = product_dataset[1]
target

/Volumes/Cucumber/Projects/item-identification/src/hcmus/data/_data_augmentation.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if isinstance(t, transforms.RandomHorizontalFlip) and random.random() < t.p:


{'boxes': tensor([[ -51.4776, -111.7351,  678.9031,  651.2979]]),
 'labels': tensor([74])}

In [9]:
product_dataloader = product_dataset.get_dataloader(shuffle=False, batch_size=4)

# 2. Finetune Model

In [10]:
import torchvision
import torch.optim as optim
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
device = "mps"

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
num_classes = len(product_dataset._labels)
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.0005, momentum=0.9, weight_decay=0.0005)

In [ ]:
model.to(device)
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, targets in product_dataloader:
        optimizer.zero_grad()
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print(f"Loss: {loss:.4f}")

print("Training Complete!")